> amortized inference

- https://zhuanlan.zhihu.com/p/368959795

- E-step: 取 $q(z)=p(z|x)$，此时 $kl(q(z)\|p(z|x))=0$
- M-step: $\arg\max \int q(z)\log p(x,z)dz$

### EM

> EM算法的精妙之处在于，它保证了每一步迭代都会使观测数据的似然函数 $L(\theta)$ 单调不减，即 $L(\theta^{(t+1)}) \ge L(\theta^{(t)})$

- 变量及符号
    - $X = \{x_1, x_2, \dots, x_m\}$：观测数据 (Observed Data)，我们能看到的数据（如身高列表）。
    - $Z = \{z_1, z_2, \dots, z_m\}$：隐变量 (Latent Variable)，我们无法观测的数据（如性别列表）。
    - $(X, Z)$：完整数据 (Complete Data)，包含了观测数据和隐变量。
    - $\theta = \{\mu_M, \sigma_M^2, \mu_F, \sigma_F^2, \pi_M, \pi_F\}$：模型参数 (Parameters)，这是我们想要估计的目标
- 目标
    - $ L(\theta) = \log P(X|\theta) =  \log \sum_{Z} P(X, Z | \theta) $
        - 对数里面带着一个求和项，这在求导和优化时非常困难
- EM：EM算法通过引入一个辅助的Q函数来巧妙地解决这个问题。(假设在第 $t$ 次迭代时，我们拥有的参数是 $\theta^{(t)}$)
    - E-Step (Expectation Step):
        - 计算完整数据对数似然 $\log P(X, Z | \theta)$ 关于 $P(Z|X, \theta^{(t)})$ 的期望。这个期望函数被称为 Q函数。
        - $Q(\theta | \theta^{(t)}) = E_{Z|X, \theta^{(t)}}[\log P(X, Z | \theta)]= \sum_{Z} P(Z|X, \theta^{(t)}) \log P(X, Z | \theta) $
            - $P(Z|X, \theta^{(t)})$ 是在给定当前观测数据 $X$ 和上一轮参数 $\theta^{(t)}$ 的条件下，隐变量 $Z$ 的后验概率。在我们的身高例子里，这就是计算每个身高 $x_i$，属于某个性别 $z_i$ 的概率。这通常是E步中实际需要计算的部分。
            - $\log P(X, Z | \theta)$ 是完整数据的对数似然。因为 $Z$ 已知，这个表达式通常很简单（比如，如果知道性别，计算高斯分布的概率就很容易）。
        - $ Q(\theta | \theta^{(t)}) = \sum_{i=1}^{m} \sum_{z_i} P(z_i|x_i, \theta^{(t)}) \log P(x_i, z_i | \theta) $
    - M-Step (Maximization Step):
        - $ \theta^{(t+1)} = \arg\max_{\theta} Q(\theta | \theta^{(t)}) $

### EM 计算示例

- https://www.youtube.com/watch?v=3zbAsgCf1Sw

- 情况一：已知分布，未知数据
    - `[1, 2, x]` 这三个样本是从一个均值为 1、方差为 1 的正态分布 $N(1, 1)$ 中抽取的。那么，对于缺失数据 x 的最佳猜测是什么？
    - 因为我们知道分布的均值是 1，所以对 x 的最佳猜测就是其期望值，即 x = 1。
- 情况二：数据完整，未知参数
    - 我告诉你 `[0, 1, 2]` 这三个样本是从一个均值为 $\mu$、方差为 1 的正态分布 $N(\mu, 1)$ 中抽取的。那么，对于未知参数 $\mu$ 的最佳猜测是什么？
    - 在这种情况下，$\mu$ 的最大似然估计就是样本均值，即 $\mu = \frac{0+1+2}{3} = 1$。
- 情况三：数据不完整且参数未知（EM 算法的核心问题）
    - 我告诉你 `[1, 2, x]` 这三个样本是从一个均值为 $\mu$、方差为 1 的正态分布 $N(\mu, 1)$ 中抽取的。那么，对于缺失数据 x 和未知参数 $\mu$ 的最佳猜测是什么？
    - 我们既不知道参数 $\mu$，也不知道缺失数据 x。如果我们知道 $\mu$，就可以像情况一那样估计 x。如果我们知道 x，就可以像情况二那样估计 $\mu$。这就形成了一个“鸡生蛋，蛋生鸡”的困境，而 EM 算法正是为了解决这类问题而设计的。

- 似然函数 (Likelihood Function)
    - 假设数据点服从正态分布 $p(x|\mu) \sim N(\mu, 1)$。对于包含缺失数据的完整数据集 (x, 1, 2)，其似然函数可以表示为：
        - $L(x, 1, 2 | \mu) = p(x|\mu) \times p(1|\mu) \times p(2|\mu)$
- EM 算法通过迭代的方式交替进行以下两步，直到收敛：
    - 初始化 (Initialization): 对未知参数 $\mu$ 进行一个初始猜测，记为 $\mu_0$。
    - E-步 (Expectation Step): 基于当前的参数估计 $\mu_0$，计算完整数据对数似然函数关于缺失数据 x 的期望。
        - $E[\log L | \mu_0] = \int p(x|\mu_0) \log L(x, 1, 2 | \mu) dx$
    - M-步 (Maximization Step): 最大化在 E-步中计算出的期望，以更新参数的估计值，得到 $\mu_1$。
        - $\mu_1 = \underset{\mu}{\operatorname{argmax}} E[\log L | \mu_0]$

- 第 0 步 (初始化): 随机猜测一个 $\mu$ 的初始值，例如 $\mu_0 = 0$。
- 第 1 步 (类 E-步): 基于 $\mu_0 = 0$，我们对缺失数据 x 的最佳猜测是其期望值，即 $x_0 = 0$。
- 第 2 步 (类 M-步): 现在我们有了一个“完整”的数据集 [1, 2, 0]。我们基于这个数据集来更新 $\mu$ 的估计值，即 $\mu_1 = \frac{1+2+0}{3} = 1$。
- 第 3 步 (类 E-步): 基于新的参数 $\mu_1 = 1$，我们再次更新对 x 的猜测，得到 $x_1 = 1$。
- 第 4 步 (类 M-步): 基于新的“完整”数据集 [1, 2, 1]，我们再次更新 $\mu$，得到 $\mu_2 = \frac{1+2+1}{3} = \frac{4}{3}$。
- ... 这个过程持续进行，$\mu$ 和 x 的值会不断更新，并逐渐逼近一个稳定点。

In [1]:
import numpy as np

In [3]:
mu = 0
for i in range(10):
    x = mu
    mu = np.mean([1, 2, x])
    print(i, mu)

0 1.0
1 1.3333333333333333
2 1.4444444444444444
3 1.4814814814814816
4 1.4938271604938274
5 1.4979423868312758
6 1.4993141289437586
7 1.4997713763145863
8 1.4999237921048623
9 1.4999745973682874


- 当算法收敛时，参数和缺失数据的估计值将达到一个平衡状态，不再发生变化。在这个稳定点上，以下两个条件必须同时满足：
    - 参数 $\mu$ 是当前完整数据的样本均值：$\mu=\frac{1+2+x}{3}$
    - 缺失数据 x 的最佳猜测是当前的参数 $\mu$：$x=\mu$
- 将第二个方程代入第一个方程，我们可以解出收敛时的值 $\mu^*$ 和 $x^*$：
    - $\mu^* = \frac{1+2+\mu^*}{3}$
    - $x^* = \mu^* = 1.5$。